# Preperation of the data
The survey results of the Stack Overflow survey of 2018 will be used. The results of the survey mainly consists of categorical answers on questions like: 'Do you write code as a hobby?' or 'How many times do you exercise each week?'. Before a Machine Learning classifier can be used, all the values with type 'object' (string) must be mapped to a numerical representation of that object. On top of that, the survey will be cleared of 'unimportant' columns that have no correlation with the target 'JobSatisfaction' and rows containing NaN values in the target column 'JobSatisfaction' will be dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from scipy import stats

### 1. Import the original Stack Overflow 2018 survey results

In [2]:
# Import original dataset
so_survey_original = pd.read_csv('./dataset/2018 Stack Overflow Survey Results.csv', low_memory=False)

print('Rows: %i, Columns: %i' % so_survey_original.shape)
so_survey_original.head(3)

Rows: 98855, Columns: 129


,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,...,3 - 4 times per week,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy
1,3,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,...,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
2,4,Yes,Yes,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",20 to 99 employees,Engineering manager;Full-stack developer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The original data frame will be (deep) copied to a new data frame on which the preperation will take place. After each preperation step the original data frame can be used to make 'before' and 'after' comparisons with the new (prepared) data frame.

In [3]:
# Copy the original data frame to a new data frame
so_survey = so_survey_original.copy(deep=True)

### 2. Drop rows with missing job satisfaction
The target feature is 'JobSatisfaction'. It isn't desired to have missing values for a target feature, because the value NaN doesn't refer to numerical classification value. All target values have to be a numerical classification value before the Machine Learning model can be made.

In [4]:
# Drop all rows with value NaN in the column 'JobSatisfaction'
mask = pd.isnull(so_survey['JobSatisfaction']) == False
so_survey = so_survey[mask].reset_index(drop=True)

print('Rows (orig): %i, Rows (prep): %i' % (so_survey_original.shape[0], so_survey.shape[0]))

Rows (orig): 98855, Rows (prep): 69276


### 3. Drop unimportant columns
Some columns can be left out because they have no correlation with the target column 'JobSatisfaction', are redundant or have too many missing values.

#### 3.1. Drop columns 'Salary' and 'Salary Type'
The column 'ConvertedSalary' is the combination of the columns 'Salary' and 'SalaryType'. The columns 'Salary' and 'SalaryType' are therefore redundant and can be dropped.

In [5]:
# Drop the columns 'Salary' and 'SalaryType'
so_survey.drop(so_survey[['Salary', 'SalaryType']], axis=1, inplace=True)

print('Columns (orig): %i, Columns (prep): %i' % (so_survey_original.shape[1], so_survey.shape[1]))

Columns (orig): 129, Columns (prep): 127


#### 3.2. Drop column 'Respondent'
The column 'Respondent' is an unique identifier for each survey respondent. The values within this column aren't correlated with the target column 'JobSatisfaction'.

In [6]:
# Drop the columns 'Respondent'
so_survey.drop(so_survey[['Respondent']], axis=1, inplace=True)

print('Columns (orig): %i, Columns (prep): %i' % (so_survey_original.shape[1], so_survey.shape[1]))

Columns (orig): 129, Columns (prep): 126


#### 3.3. Drop columns containing 'StackOverflow' and 'Survey'
Questions containing 'StackOverflow' and 'Survey' are used to gather feedback for Stack Overflow. The results of these questions aren't correlated with the targed column 'JobSatisfaction'.

In [7]:
def find_columns(df, str_list):
    """ Find all columns which contain
    any of the strings in 'str_list'
    in their name.
    """
    columns = []
    for column in so_survey.columns:
        # Check for every string in 'str_list' 
        # if it's in name of the column.
        for s in str_list:
            if s in column:
                columns.append(column)
                # Break prevents duplicate column names in the list 'columns'
                break
    return columns

In [8]:
columns = find_columns(so_survey, ['StackOverflow', 'Survey'])
so_survey.drop(so_survey[columns], axis=1, inplace=True)

print('Columns (orig): %i, Columns (prep): %i' % (so_survey_original.shape[1], so_survey.shape[1]))

Columns (orig): 129, Columns (prep): 116


#### 3.4. Drop columns 'TimeAfterBootcamp' and 'MilitaryUS'
Both the columns 'TimeAfterBootcamp' and 'MilitaryUS' have more than 80% of missing data. Imputing statistical values (e.g. mean, mode, etc.) won't result in accurate data.

In [9]:
# Drop the columns 'SalaTimeAfterBootcampry' and 'MilitaryUS'
so_survey.drop(so_survey[['TimeAfterBootcamp', 'MilitaryUS']], axis=1, inplace=True)

print('Columns (orig): %i, Columns (prep): %i' % (so_survey_original.shape[1], so_survey.shape[1]))

Columns (orig): 129, Columns (prep): 114


### 4. Prepare values in list format
Some values in the dataset are denoted as $'Python;Java;C#'$. This format represents a list of different values. First, the ';' seperated list has to be converted to a real list data structure. Second, the list needs to be splitted into different columns to make it easy interpretable for Machine Learning algorithms. 

#### 4.1. Find columns containing list format
The columns containing ';' seperated lists will be searched and listed below to get an overview.

In [10]:
def find_cols_containing_char(df, char):
    """ Finds all columns with values
    containing the given character.
    """
    list_columns = []
    # Look for every column
    for col in df.columns:
        values = df[col].values
        # Numerical columns can be skipped, because columns 
        # with type 'number' doesn't contain(string-like) characters.
        if np.issubdtype(values.dtype, np.number) == False:
            # Look for every value in column
            for val in values:
                if char in str(val):
                    list_columns.append(col)
                    # Break prevents extra time spent while this 
                    # column already contains the character
                    break
    return list_columns

In [11]:
# Find all columns with values containing the character ';'
list_columns = find_cols_containing_char(so_survey, ';')
list_columns

['DevType',
 'CommunicationTools',
 'EducationTypes',
 'SelfTaughtTypes',
 'HackathonReasons',
 'LanguageWorkedWith',
 'LanguageDesireNextYear',
 'DatabaseWorkedWith',
 'DatabaseDesireNextYear',
 'PlatformWorkedWith',
 'PlatformDesireNextYear',
 'FrameworkWorkedWith',
 'FrameworkDesireNextYear',
 'IDE',
 'Methodology',
 'VersionControl',
 'AdBlockerReasons',
 'AdsActions',
 'ErgonomicDevices',
 'Gender',
 'SexualOrientation',
 'RaceEthnicity']

#### 4.2. Convert ';' seperated values to binarized representation
List values such as $'Python;Java;C#'$ can't be used as input for a Machine Learning algorithm. First, the value has to be numerical. Second, numerification of the ';' values as is will result a unique class for every unique list. It is instead needed to get a unique class for every language present in the list.<br><br>
To do this the the *MultiLabelBinarizer* algorithm from the *sklearn.preprocessing* library can be used. This algorithm converts values like $[['Python', 'Java', 'C#'], ['Python', 'C#']]$ to the following representation: $[[1, 1, 1], [1, 0, 1]]$ with the columns 'Python', 'Java' and 'C#'. This way the the values will be converted to a $0$ or a $1$ for each row, which is easier interpretable for a Machine Learning algorithm.

In [12]:
def binarize_list_columns(df, columns, sep=';'):
    """ Converts all ';' (or other seperator) lists to actual list data structures.
    The newly created list will be converted to a binarized matrix,
    in which each column is a unique value present in the list.
    The column tells if a certain value is present in the corresponding row or not
    by using boolean (binary) values.
    """
    df_copy = df.copy(deep=True)
    mlb = MultiLabelBinarizer()
    for col in columns:
        # Convert list formats like 'Python;Java;C#' to [Python, Java, C#]
        # If value is NaN, the value will be converted to []
        tranformed_vals = [x.split(';') if x is not np.nan else [] for x in df_copy[col].values]
        
        # Binerize the tranformed values
        binerized = mlb.fit_transform(tranformed_vals)

        # Add the binerized value to the existing data frame
        df_copy = df_copy.join(pd.DataFrame(binerized, columns=mlb.classes_), lsuffix='_left', rsuffix='_right')
        
        # Delete original column because it doesn't add value anymore
        del df_copy[col]
    return df_copy

In [13]:
# The column 'Student' will colide with the value 'Student' in the binarization process.
# Therefore the column 'Student' will be renamed to 'is_student'
so_survey.rename(columns={'Student': 'is_student'}, inplace=True)

In [14]:
# Start binarization of list columns
so_survey_bin = binarize_list_columns(so_survey, list_columns, ';')

print('Columns (orig): %i, Columns (prep): %i' % (so_survey_original.shape[1], so_survey_bin.shape[1]))
so_survey_bin.head(3)

Columns (orig): 129, Columns (prep): 410


,Hobby,OpenSource,Country,is_student,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,YearsCodingProf,...,Bisexual or Queer,Gay or Lesbian,Straight or heterosexual,Black or of African descent,East Asian,Hispanic or Latino/Latina,Middle Eastern,"Native American, Pacific Islander, or Indigenous Australian",South Asian,White or of European descent
0,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,3-5 years,3-5 years,...,0,0,1,1,0,0,0,0,0,0
1,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",30 or more years,18-20 years,...,0,0,1,0,0,0,0,0,0,1
2,Yes,Yes,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",20 to 99 employees,24-26 years,6-8 years,...,0,0,0,0,0,0,0,0,0,0


### 5. Convert string categories to numerical categories
Most values in the survey results are  in text-format. Text isn't easy to interpret for Machine Learning algorithm. To solve this problem all text-formatted values will be converted by numerical values which will denote the class the value belongs to.

#### 5.1. Find columns with string values
Let's find out how many columns with text-format are present in the dataset, to get a better understanding of the data.

In [15]:
def find_object_cols(df):
    """ Finds all columns that 
    have the dtype of 'object'
    """
    return [x for x in df.columns if df[x].dtype == object]

In [16]:
# Find all columns with values in text-format
obj_cols = find_object_cols(so_survey_bin)

print('Number of columns with object type: %i' % (len(obj_cols)))

Number of columns with object type: 51


#### 5.2. Convert string values to numerical values
The value containing string will be converted to numerical value. E.g. a column with the values 'yes' and 'no' will be mapped to 1 (yes) and 0 (no).

In [17]:
def encode_labels(df, columns):
    """ Maps all values in the given columns
    to a numeric value. Values with the same
    text, will get the same numeric value/class.
    """
    df_copy = df.copy(deep=True)
    maps = {}
    # Convert values in all of the given columns
    for col in columns:
        # Map values in a dictionary.
        # E.g. column 'Language' with unique values ['Python', 'Java', C#]
        # will get mapped to 'Python': 0, 'Java': 1, 'C#': 2.
        maps[col] = {key: value for value, key in enumerate(df[col].unique()) if key is not np.nan}
        
    # Use the mappings to replace all the values 
    # to their mapped numerical values in one go.
    df_copy.replace(maps, inplace=True)
    
    # Return the encoded data frame and the mappings for decode purposes.
    return df_copy, maps

In [18]:
# Encode/map values to a numerical value
so_survey_conv, maps = encode_labels(so_survey_bin, obj_cols)

print('Number of columns with object type: %i' % (len(find_object_cols(so_survey_conv))))
so_survey_conv.head(3)

Number of columns with object type: 0


,Hobby,OpenSource,Country,is_student,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,YearsCodingProf,...,Bisexual or Queer,Gay or Lesbian,Straight or heterosexual,Black or of African descent,East Asian,Hispanic or Latino/Latina,Middle Eastern,"Native American, Pacific Islander, or Indigenous Australian",South Asian,White or of European descent
0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0,...,0,0,1,1,0,0,0,0,0,0
1,0,1,1,0.0,1,0.0,1.0,1.0,1.0,1,...,0,0,1,0,0,0,0,0,0,1
2,0,1,2,0.0,1,1.0,2.0,0.0,2.0,2,...,0,0,0,0,0,0,0,0,0,0


### 6. Export prepped data frame
The preprocessed data frame can be exported to a csv file, so future Machine Learning algorithms can use it.

In [19]:
# Export preprocessed data frame to a csv file
so_survey_conv.to_csv('./dataset/so_survey_prepped.csv', index=False)

### 7. Export used mappings
The used mappings to numerical values are useful for decoding. A data frame will be created in which the numerical values are the row indeces and the text values are the column indeces. This will make future decoding of the values a lot easier.

In [20]:
def inverse_mappings(maps):
    """ Switches the keys and values for every key value pairs of the original mappings.
    E.g. 'Hobby': { 'yes': 1, 'no': 0 },
    will become 'Hobby': { 1: 'yes', 0: 'no' }
    """
    maps_inv = {}
    # Look for every key/column with mapped values
    for key in maps.keys():
        m = {}
        # Switch keys and values for all key value pairs.
        for k, v in maps[key].items():
            m[v] = k
        maps_inv[key] = m
    return maps_inv

In [21]:
# Inverse the current mappings and create a mappings matrix dataframe from the inversed mappings
maps_matrix = pd.DataFrame(inverse_mappings(maps))

maps_matrix.head(3)

,Hobby,OpenSource,Country,is_student,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,YearsCodingProf,...,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,Exercise,EducationParents,Age,Dependents
0,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,3-5 years,3-5 years,...,Extremely interested,Extremely interested,Between 5:00 - 6:00 AM,9 - 12 hours,1 - 2 hours,Never,3 - 4 times per week,"Bachelor’s degree (BA, BS, B.Eng., etc.)",25 - 34 years old,Yes
1,No,Yes,United Kingdom,"Yes, part-time",Employed full-time,Associate degree,"A natural science (ex. biology, chemistry, phy...","10,000 or more employees",30 or more years,18-20 years,...,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,NaN,Daily or almost every day,NaN,35 - 44 years old,NaN
2,NaN,NaN,United States,"Yes, full-time","Independent contractor, freelancer, or self-em...",Some college/university study without earning ...,"Computer science, computer engineering, or sof...",100 to 499 employees,24-26 years,6-8 years,...,NaN,NaN,NaN,NaN,NaN,3 - 4 times per week,NaN,Some college/university study without earning ...,NaN,No


In [22]:
# Export the mappings matrix to a csv file
maps_matrix.to_csv('./dataset/so_survey_mappings.csv', index=False)

### 8. Impute missing data
A lot of data is missing, this missing data can be imputed with statistical values (e.g. mean, mode, etc.).

#### 8.1. Impute missing values for numerical and categorical columns
Only the column 'ConvertedSalary' is of the numerical data type. We can impute the missing data of the numerical data type by using the mean of all values. All the other columns are of the categorical data type. We can impute the missing data of the categorical data type by using the mode (most frequent) of all values.

In [23]:
def impute_missing(df):
    """ Missing values of numerical columns will 
    be replaced by the mean of all values of that column.
    Missing values of categorical columns will
    be replaced by the mode of all values of that column.
    """
    # Look for every column
    for col in df.columns:
        values = df[col].values
        stat_value = 0
        if col is 'ConvertedSalary':
            # If column is numerical: get mean of all values
            stat_value = values[~np.isnan(values)].mean()
        else:
            # If column is numerical: get mode of all values
            stat_value = stats.mode(values[~np.isnan(values)])[0][0]
        # Fill missing values with statistical value
        df[col].fillna(stat_value, inplace=True)

In [24]:
# Impute the missing values with statistical values (mean, mode, etc.)
impute_missing(so_survey_conv)

### 9. Export imputed data frame
The imputed preprocessed data frame will also be exported to a csv file. This is because both the imputed version and the not-imputed version of the preprocessed dataset will be tested in the project. The one that will perform better will be chosen.

In [25]:
# Export preprocessed imputed data frame to a csv file
so_survey_conv.to_csv('./dataset/so_survey_prepped_imputed.csv', index=False)